In [117]:
!pip install numpy 
!pip install pandas
!pip install tensorflow

In [118]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [119]:
import pandas as pd
import numpy as np


In [4]:
!ls -l /content/gdrive/

total 8
drwx------ 30 root root 4096 Jun 23 19:49 MyDrive
drwx------  2 root root 4096 Jun 23 19:49 Shareddrives


In [120]:
filepath = '/content/gdrive/MyDrive/IA/dataset/DIAGNOSES_ICD.csv'
dataset_dig9 = pd.read_csv(filepath, sep=',', decimal='.')

In [121]:
def get_unique( data , columm ):
  uniq_list = data[columm].unique().tolist()
  return uniq_list

In [122]:
diagnoses = get_unique(data=dataset_dig9 , columm='icd9_code')
patients  = get_unique(data=dataset_dig9 , columm='subject_id')
admitions = get_unique(data=dataset_dig9 , columm='hadm_id')
print(diagnoses)

['99591', '99662', '5672', '40391', '42731', '4280', '4241', '4240', '2874', '03819', '7850', 'E8791', 'V090', '56211', '28529', '25000', 'V5867', 'E9342', '41401', '2749', '3051', '570', '07030', '07054', '30401', '2875', '2760', '0389', '41071', '78551', '486', '20280', '4582', '2724', '81201', '4928', '8028', '8024', '99812', '41511', '2851', 'E8859', 'E8788', 'V1259', '4019', '51881', '5770', '30390', '5781', '5845', '2848', '5722', '78559', '99592', '5711', '49390', '7299', '431', '4010', '5990', '2761', 'V103', '3569', '2859', '3970', '4266', '5859', '2948', '5691', '79092', '0380', '99673', '5761', '57451', '5856', '4538', '78552', '51882', '5119', '99659', '7211', 'V1588', '2380', '2930', '0414', 'V4502', 'V1046', '81249', '1985', '5070', '42830', '2765', '5849', '24220', '51919', '2767', 'E8889', '2720', '311', '43811', '53390', '43310', '6000', '412', '2762', '2554', '53081', '3310', '29410', '725', '2449', '99667', '71105', '2763', '99677', '7837', '45981', 'E8781', 'E8498',

In [8]:
len(patients)

100

In [123]:
'''
Pacienteid: { 
   admitions: []
   diagnoses: [] 
}
'''
Mapdict = {}
for patient in patients: 
  Mapdict[str(patient)] = {'admitions':dataset_dig9.loc[dataset_dig9['subject_id'] == patient]['hadm_id'].unique().tolist() , 'diagnoses':dataset_dig9.loc[dataset_dig9['subject_id'] == patient]['icd9_code'].unique().tolist()}

In [125]:
print(Mapdict)

{'10006': {'admitions': [142345], 'diagnoses': ['99591', '99662', '5672', '40391', '42731', '4280', '4241', '4240', '2874', '03819', '7850', 'E8791', 'V090', '56211', '28529', '25000', 'V5867', 'E9342', '41401', '2749', '3051']}, '10011': {'admitions': [105331], 'diagnoses': ['570', '07030', '07054', '30401', '2875', '2760']}, '10013': {'admitions': [165520], 'diagnoses': ['0389', '41071', '78551', '486', '42731', '20280', '4240', '4582', '2724']}, '10017': {'admitions': [199207], 'diagnoses': ['81201', '4928', '8028', '8024', '99812', '41511', '2851', '486', 'E8859', '25000', 'E8788', 'V1259', '2724', '4019']}, '10019': {'admitions': [177759], 'diagnoses': ['0389', '51881', '5770', '30390', '5781', '5845', '2848', '5722', '78559', '99592', '5711', '4019', '49390', '7299']}, '10026': {'admitions': [103770], 'diagnoses': ['431', '4010', '5990', '2761', 'V103', '3569', '2859']}, '10027': {'admitions': [199395], 'diagnoses': ['4280', '4240', '3970', '4266', '42731', '5859', '41401', '4019

In [124]:
diagnoses.sort()

In [11]:
''' 
 \  D0  D1 ... Dn 
 P0 1   0  ... 0
 P1 0   0  ... 1
 ... 
 Pn 1   1  ... 1
 '''
Bigvet = []
for patient in patients: 
  patVet = [] 
  for diagnostic in diagnoses:
    if diagnostic in Mapdict[str(patient)]['diagnoses']: 
      patVet.append(1)
    else: 
      patVet.append(0)
  Bigvet.append(patVet) 

In [12]:
Nw = pd.DataFrame(Bigvet)
Nw.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580
count,100.000000,100.000000,100.000000,100.000000,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,...,100.00,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.00,100.00,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.00,100.000000,100.00,100.000000,100.000000,100.000000
mean,0.070000,0.020000,0.020000,0.020000,0.01,0.020000,0.01,0.01,0.01,0.020000,0.01,0.160000,0.01,0.01,0.01,0.020000,0.01,0.08000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.020000,0.020000,0.01,0.01,0.01,0.030000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.030000,0.040000,0.020000,0.050000,0.01,0.01,0.01,0.01,0.01,0.01,0.020000,0.01,0.01,0.01,0.020000,0.01,0.01,0.01,0.01,0.01,0.070000,0.040000,0.020000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.090000,0.01,0.050000,0.01,0.030000,0.030000,0.020000
std,0.256432,0.140705,0.140705,0.140705,0.10,0.140705,0.10,0.10,0.10,0.140705,0.10,0.368453,0.10,0.10,0.10,0.140705,0.10,0.27266,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.140705,0.140705,0.10,0.10,0.10,0.171447,0.10,0.10,0.10,0.10,0.10,0.10,0.10,...,0.10,0.171447,0.196946,0.140705,0.219043,0.10,0.10,0.10,0.10,0.10,0.10,0.140705,0.10,0.10,0.10,0.140705,0.10,0.10,0.10,0.10,0.10,0.256432,0.196946,0.140705,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.287623,0.10,0.219043,0.10,0.171447,0.171447,0.140705
min,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0

In [23]:
def noise(size): 
  return np.random.random((size,581))

In [126]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import matplotlib.pyplot as plt

In [ ]:
''' 
x = [0 1 .. 1]
Dis(x) = [p1 ... pn]
Z = [1 1 .. 1] noise vector
gen(Z) = [1 0 .. 1]  
Dis(gen(z)) = [^p1 ^p2 ... ^pn]
'''

' \nx = [0 1 .. 1]\nDis(x) = [p1 ... pn]\nZ = [1 1 .. 1] noise vector\ngen(Z) = [1 0 .. 1]  \nDis(gen(z)) = [^p1 ^p2 ... ^pn]\n'

In [127]:
def generator(hsize=[10, 10],reuse=False):
    # Cria o modelo / Funciona como stack de opercoes
    model =  tf.keras.Sequential()

    # Cria primeira camada da rede neural usando os input Z / layer.dense cria uma rede neural do tipo densa 
    model.add(tf.keras.layers.Dense(hsize[1],name='GL1' ,activation=tf.keras.layers.LeakyReLU(),)) #input_shape=(BATCH_SIZE,581)))

    # Cria a segunda camada da rede usando o input h1(Z)
    model.add( tf.keras.layers.Dense(hsize[1],name='GL2',activation=tf.keras.layers.LeakyReLU()))

    # Cria a de saida camada da rede usando o input h2(h1(Z))
    model.add(tf.keras.layers.Dense(581,name='GLout', activation='sigmoid')) 
    return model

In [128]:
def discriminator(hsize=[10, 10],reuse=False):
    # Cria o modelo / Funciona como stack de opercoes
    model =  tf.keras.Sequential()
    # Cria primeira camada da rede neural usando os input Z / layer.dense cria uma rede neural do tipo densa 
    model.add(tf.keras.layers.Dense(hsize[0],name='DL1'  ,activation='sigmoid')) #,input_shape=(BATCH_SIZE,581)))
    # Cria a segunda camada da rede usando o input h1(Z)
    model.add( tf.keras.layers.Dense(hsize[1], name='DL2',activation=tf.keras.layers.LeakyReLU()))
    model.add( tf.keras.layers.Dense(hsize[1],name='DL3', activation=tf.keras.layers.LeakyReLU()))
    # Cria a de saida camada da rede usando o input h2(h1(Z))
    model.add(tf.keras.layers.Dense(2, name='DL4'))
    # Cria a de saida camada da rede usando o input h3(h2(h1(Z)))
    model.add(tf.keras.layers.Dense(1,name='DLout', activation='sigmoid'))
    return model

In [129]:
EPOCHS = 10000
BATCH_SIZE = 15

In [130]:
gen = generator()
dis = discriminator()

In [131]:
Z = noise(15)
print(Z.shape)

(15, 581)


In [132]:
fake_data = gen(Z, training=False)
print(fake_data)

tf.Tensor(
[[0.5057122  0.52418995 0.4939573  ... 0.49284056 0.48201203 0.5024631 ]
 [0.49437878 0.51176995 0.49984506 ... 0.47084293 0.4921109  0.49899396]
 [0.50004077 0.5320632  0.50224644 ... 0.48220122 0.46400326 0.5033616 ]
 ...
 [0.49742982 0.5130205  0.50205004 ... 0.5000495  0.46358562 0.5155095 ]
 [0.49551952 0.50906783 0.50295454 ... 0.50135016 0.47592315 0.5110592 ]
 [0.5074448  0.52855384 0.49641347 ... 0.4762241  0.5073304  0.48301378]], shape=(15, 581), dtype=float32)


In [133]:
decision = dis(fake_data, training=False)

In [134]:
print(decision)

tf.Tensor(
[[0.5099108 ]
 [0.50974786]
 [0.5097241 ]
 [0.5097573 ]
 [0.5096154 ]
 [0.5095458 ]
 [0.51005656]
 [0.50968945]
 [0.5096953 ]
 [0.5098569 ]
 [0.50978947]
 [0.5097824 ]
 [0.50978535]
 [0.50983   ]
 [0.50994545]], shape=(15, 1), dtype=float32)


In [135]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [136]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [137]:
# Otimizador
generator_optimizer = tf.keras.optimizers.Adam(1e-3,)
#generator_optimizer =tf.keras.optimizers.RMSprop(1e-3)

discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)
#discriminator_optimizer = tf.keras.optimizers.RMSprop(1e-4)

In [138]:
epoch_vec=[]
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [139]:
from random import randint

In [140]:
def sample_data(matrix,batch):
  sample =[]
  for integer in range(batch): 
    sample.append(matrix[randint(0,len(matrix)-1)])
  #print(sample)
  return np.array(sample)

In [142]:
@tf.function
def train_step(Y):
    Z = noise(BATCH_SIZE) #
    with tf.GradientTape(persistent=True) as tape:
      #print('Entrei no loop')
      fake_data = gen(Z, training=True)  
      real_output = dis(Y, training=True)
      fake_output = dis(fake_data, training=True)
      #print(gen.)
      gen_loss = generator_loss(fake_output)
      #print(gen_loss)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = tape.gradient(gen_loss, gen.trainable_variables)
    gradients_of_discriminator = tape.gradient(disc_loss, dis.trainable_variables)
    del tape
    generator_optimizer.apply_gradients(zip(gradients_of_generator, gen.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, dis.trainable_variables))


In [143]:
epoch_vec = []
for x in range(EPOCHS):
    Y = sample_data(Bigvet,batch= BATCH_SIZE)
    train_step(Y)
    #gen.summary()
    #dis.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


In [144]:
z = noise(BATCH_SIZE)
fake = gen(Z, training=False)
print(z)
print(fake)

[[0.519168   0.20877213 0.37925262 ... 0.62934597 0.84784484 0.45168483]
 [0.32664355 0.6925659  0.89269318 ... 0.68199402 0.75771055 0.30760834]
 [0.45429562 0.43150156 0.84157625 ... 0.24133683 0.87339318 0.66704762]
 ...
 [0.0855073  0.60876694 0.80517502 ... 0.89396242 0.91683752 0.18732148]
 [0.32498888 0.53610196 0.17747824 ... 0.50328292 0.16079052 0.7582672 ]
 [0.82155149 0.84048205 0.76006413 ... 0.28468098 0.28981567 0.68835907]]
tf.Tensor(
[[4.4705884e-12 1.0000000e+00 1.5926004e-06 ... 1.0000000e+00
  3.7864755e-16 4.8626421e-06]
 [3.5530238e-12 1.0000000e+00 1.4325361e-06 ... 1.0000000e+00
  2.7198363e-16 3.9859174e-06]
 [4.2530532e-12 1.0000000e+00 1.1577135e-06 ... 1.0000000e+00
  3.2164221e-16 3.3235974e-06]
 ...
 [1.0312497e-11 1.0000000e+00 2.5524616e-06 ... 1.0000000e+00
  1.0791315e-15 8.2156021e-06]
 [6.6255768e-12 1.0000000e+00 1.8600198e-06 ... 1.0000000e+00
  6.1836836e-16 5.1380653e-06]
 [2.3765998e-12 1.0000000e+00 1.2094334e-06 ... 1.0000000e+00
  1.6754124e-

In [145]:
print(dis(fake, training=False))

tf.Tensor(
[[1.4166901e-05]
 [1.4166901e-05]
 [1.4166914e-05]
 [1.4166914e-05]
 [1.4166887e-05]
 [1.4166928e-05]
 [1.4166914e-05]
 [1.4166928e-05]
 [1.4166913e-05]
 [1.4166913e-05]
 [1.4166901e-05]
 [1.4166860e-05]
 [1.4166860e-05]
 [1.4166901e-05]
 [1.4166913e-05]], shape=(15, 1), dtype=float32)
